In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /home/sinhaa2/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sinhaa2/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
data = pd.read_csv("spam.csv", encoding='ISO-8859-1') 
data = data[['v1', 'v2']]
data = data.rename(columns={"v1": "IsSpam", "v2": "Message"}, errors="raise")
data.loc[data["IsSpam"] == "spam", "IsSpam"] = 1
data.loc[data["IsSpam"] == "ham", "IsSpam"] = 0

data["Tokens"] = data["Message"].apply(nltk.word_tokenize)
stop = stopwords.words('english')
data["Tokens"] = data["Tokens"].apply(lambda x: [item for item in x if item not in stop])
# tokenList = list(set([item for sublist in data["Tokens"].tolist() for item in sublist]))
# print(len(tokenList))

In [4]:
tokenList = list(set([item for sublist in data["Tokens"].tolist() for item in sublist]))
tokenData = pd.DataFrame(tokenList, columns = ["Words"])
tokenData["nSpam"] = 0
tokenData["nHam"] = 0

print(data.shape)
for index, row in data[["IsSpam", "Tokens"]].iterrows():
    columnName = "nSpam" if row.IsSpam == 1 else "nHam"
    for token in row["Tokens"]:
        tokenData.loc[tokenData["Words"] == token, columnName] += 1

(5572, 3)


In [18]:
# tokenData["Difference"] = tokenData["nSpam"] - tokenData["nHam"]
# tokenData.sort_values(by=["Difference"], ascending=False).head(10)

tokenData["Probability"] = (tokenData["nSpam"] - tokenData["nHam"]) / (tokenData["nSpam"] + tokenData["nHam"])
tokenData.sort_values(by=["Probability"], ascending=False).head(10)

,Words,nSpam,nHam,Probability,Difference
11766,ree,1,0,1.0,1
7759,handset,4,0,1.0,4
5666,08717507382,1,0,1.0,1
2666,November,1,0,1.0,1
2667,ZOUK,1,0,1.0,1
2672,0776xxxxxxx,2,0,1.0,2
9920,eg,11,0,1.0,11
5662,Code:4xx26,1,0,1.0,1
7841,07753741225,1,0,1.0,1
2680,09066368327,1,0,1.0,1


In [9]:
identifiers = data.copy()
identifiers = data.rename(columns={"Message": "Original_Message"}, errors="raise")

for item in tokenData["Words"].tolist():
    identifiers[item] = 0

In [10]:
for item in tokenData["Words"].tolist():
#     print(item)
    identifiers.loc[identifiers["Original_Message"].str.contains(item, regex=False), item] = 1

In [11]:
isSpam = pd.to_numeric(identifiers["IsSpam"])
identfiers_matrix = identifiers.drop(["Original_Message", "Tokens", "IsSpam"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(identfiers_matrix, isSpam, test_size=0.2, random_state=0)

In [17]:
gnb = GaussianNB()
model = gnb.fit(X_train, y_train)
result = model.predict(X_test)
success = sum(y_test.array == result)/X_test.shape[0]
success

0.9210762331838565

In [16]:
clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
success = sum(y_test.array == predictions)/X_test.shape[0]
success

0.8708520179372198

In [20]:
identfiers_matrix

,delicious,looked,grief,golden,parents,reason..,SPK,Loans,colours,crave,...,arithmetic,taking,kiss,Going,stuff42moro,SPTV,Promo,4882,heads,ree
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
